In [1]:
!pip install shapely
!pip install geopandas
!pip install rasterio
!pip install gdown
!pip install rasterstats
!pip install geemap

In [4]:
import geemap, ee
import pandas as pd
import geopandas as gpd
import importlib
import sys
import subprocess

In [5]:
ee.Authenticate()
ee.Initialize()

In [6]:
# Create a map
Map = geemap.Map()

# Define the coordinates and zoom level for the region of interest (ROI)
roi_longitude = 26.8
roi_latitude = 37.8
zoom_level = 11

# Define the Area of Interest (AOI) as a Polygon.
AOI = ee.Geometry.Polygon([[27.099800633717916,37.75135777501414],
[27.099800633717916,37.89454805937824],
[26.55185751848354,37.89454805937824],
[26.55185751848354,37.75135777501414]])
AOI

# Define the study area and date range
study_area = ee.Geometry.Point(roi_longitude, roi_latitude)
start_date = '2023-04-01'
end_date = '2023-07-01'

# Center the map on the ROI
Map.setCenter(roi_longitude, roi_latitude, zoom_level)

def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

# Load Sentinel-2 images and apply cloud masking, further we select only imagery with less than 5% cloud
dataset = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                 .filterBounds(study_area)
                 .filterDate(start_date, end_date)
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
                 .map(maskS2clouds)
                 .map(lambda image: image.clip(AOI)))

# Compute the median of the image collection (this only applies to the bands (not yet NDVI or NDWI))
image_median = dataset.reduce(ee.Reducer.median())

# Function to calculate the Normalized Difference Vegetation Index (NDVI) with the NIR (B8), Red (B4)
def calculateNDVI(image):
    return image.normalizedDifference(['B8_median','B4_median']).rename('ndvi')

# Function to calculate the Normalized Difference Water Index (NDWI) with the Green (B3), Nir (B8)
def calculateNDWI(image):
    return image.normalizedDifference(['B3_median','B8_median']).rename('ndwi')

#Calculate NDVI
ndvi_image = calculateNDVI(image_median)

#Calculate NDWI
ndwi_image = calculateNDWI(image_median)

# Define visualization parameters
visualization = {
    'max': 0.3,
    'bands': ['B8_median','B4_median', 'B3_median'],
}

# Define a threshold value for NDVI to classify land areas
ndvi_threshold = 0.05
ndwi_threshold = 0

vegetation_mask = ndvi_image.lte(ndvi_threshold)
water_mask = ndwi_image.gte(ndwi_threshold)

# Combine the vegetation and water masks
combined_mask = vegetation_mask.Or(water_mask)

# Apply a median filter to the combined mask to reduce noise (adjust window size as needed)
smoothed_mask = combined_mask.focal_median(radius=4)

#image_median.addBands(ndvi_image, ndwi_image)

# Mask the land from the median composite image using the combined mask
image_median_water = image_median.updateMask(smoothed_mask)

# Add layers to the map
Map.addLayer(image_median_water, visualization, 'S2 Mask')
Map

Map(center=[37.8, 26.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…